In [1]:
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from api_keys_ml import api_key

In [2]:
nasdaq_url = 'https://financialmodelingprep.com/api/v3/nasdaq_constituent?apikey=' + api_key

In [3]:
nasdaq_list_response = requests.get(nasdaq_url).json()

In [4]:
# collect a list of the NASDAQ constituents
nasdaq_companies = []
company_tickers = []

for company in nasdaq_list_response:
    nasdaq_companies.append(company['name'])
    company_tickers.append(company['symbol'])

In [5]:
data = {'Company': company_tickers, 'Q1 P/E': ['' for i in range(len(company_tickers))], 
        'Q2 P/E': ['' for i in range(len(company_tickers))], 
        'Q3 P/E': ['' for i in range(len(company_tickers))], 
        'Q1 ROE': ['' for i in range(len(company_tickers))], 
        'Q2 ROE': ['' for i in range(len(company_tickers))], 
        'Q3 ROE': ['' for i in range(len(company_tickers))], 
        'Q1 Date': ['' for i in range(len(company_tickers))],
        'Q2 Date': ['' for i in range(len(company_tickers))],
        'Q3 Date': ['' for i in range(len(company_tickers))],
       }
nasdaq_df = pd.DataFrame(data=data)
nasdaq_df.head()

,Company,Q1 P/E,Q2 P/E,Q3 P/E,Q1 ROE,Q2 ROE,Q3 ROE,Q1 Date,Q2 Date,Q3 Date
0,DOCU,,,,,,,,,
1,ZM,,,,,,,,,
2,DXCM,,,,,,,,,
3,SPLK,,,,,,,,,
4,SGEN,,,,,,,,,


In [6]:
# gather data on price/earnings ratio AND returns on equity for each company for the last three quarters
for index, row in nasdaq_df.iterrows():
    ticker = row['Company']
    quarterly_ratio_url = f'https://financialmodelingprep.com/api/v3/ratios/{ticker}?period=quarter&limit=3&apikey={api_key}' 
    response = requests.get(quarterly_ratio_url).json()

    try:
        nasdaq_df.loc[index, 'Q3 P/E'] = response[0]['priceEarningsRatio']
        nasdaq_df.loc[index, 'Q2 P/E'] = response[1]['priceEarningsRatio']
        nasdaq_df.loc[index, 'Q1 P/E'] = response[2]['priceEarningsRatio']
        nasdaq_df.loc[index, 'Q3 ROE'] = response[0]['returnOnEquity']
        nasdaq_df.loc[index, 'Q2 ROE'] = response[1]['returnOnEquity']
        nasdaq_df.loc[index, 'Q1 ROE'] = response[2]['returnOnEquity']
        nasdaq_df.loc[index, 'Q3 Date'] = response[0]['date']
        nasdaq_df.loc[index, 'Q2 Date'] = response[1]['date']
        nasdaq_df.loc[index, 'Q1 Date'] = response[2]['date']
    except IndexError:
        print('Missing NASDAQ information')

    
    

In [7]:
nasdaq_df.head()

,Company,Q1 P/E,Q2 P/E,Q3 P/E,Q1 ROE,Q2 ROE,Q3 ROE,Q1 Date,Q2 Date,Q3 Date
0,DOCU,-299.597,-400.949,-620.874,-0.086774,-0.0914405,-0.128634,2020-01-31,2020-04-30,2020-07-31
1,ZM,1191.1,1473.69,405.682,0.0183933,0.0300289,0.155144,2020-01-31,2020-04-30,2020-07-31
2,DXCM,218.504,1273.28,849.326,0.105031,0.0212948,0.0341975,2019-12-31,2020-03-31,2020-06-30
3,SPLK,-1064.43,-72.3593,-127.625,-0.0113672,-0.168823,-0.156529,2020-01-31,2020-04-30,2020-07-31
4,SGEN,759.557,-118.085,-1390.52,0.0137666,-0.0953785,-0.0116809,2019-12-31,2020-03-31,2020-06-30


In [8]:
nasdaq_df.to_csv('../output/nasdaq_financial_ratios.csv')